# Figure 1

- **a:** gmxapi command-line operation
- **b:** gmxapi command on ensemble input
- **c:** looping and custom operations

Funnelweb spider peptide structure files from
* Sorin and Pande, 2005; doi:10.1529/biophysj.104.051938
(used with permission).

Note: MPI ensemble is not currently feasible in a Jupyter notebook.

In [ ]:
import os
from pathlib import Path

In [ ]:
# Assumes the tutorial material is in `$HOME/gmxapi-introduction`, as with the Docker image.
# Otherwise, you must set PROJECT_DIR environment variable or edit the following.
project_dir = Path(os.getenv('HOME')) / 'gmxapi-introduction'
if not project_dir.exists():
    project_dir = Path(os.getenv('PROJECT_DIR', Path(os.getcwd())))
while not (project_dir / 'input_files' / 'fs-peptide').exists() and project_dir.parent != project_dir:
    project_dir = project_dir.parent
input_dir = project_dir / 'input_files' / 'fs-peptide'
assert input_dir.exists()

In [ ]:
# Import gmxapi package
import gmxapi as gmx
print(f'gmxapi Python package version {gmx.__version__}')

## Figure 1a: gmxapi command-line operation
Prepare a molecular model from a PDB file using the `pdb2gmx` GROMACS tool.

In [ ]:
# Confirm inputs exist
input_dir = project_dir / 'input' / 'fs-peptide'
assert (input_dir / 'start0.pdb').exists()

In [ ]:
# Figure 1a code
args = ['pdb2gmx', '-ff', 'amber99sb-ildn', '-water', 'tip3p']
input_files = {'-f': os.path.join(input_dir, 'start0.pdb')}
output_files = {
        '-p': 'topol.top',
        '-i': 'posre.itp',
        '-o': 'conf.gro'}
make_top = gmx.commandline_operation('gmx', args, input_files, output_files)

## Figure 1b: gmxapi command on ensemble input
Call the GROMACS MD preprocessor to create a simulation input file. Declare an ensemble simulation workflow starting from the single input file.

Note: MPI ensemble not represented in Jupyter notebook

In [ ]:
# Confirm inputs exist.
make_top.run()
assert os.path.exists(make_top.output.file['-o'].result())
assert os.path.exists(make_top.output.file['-p'].result())
cmd_dir = input_dir
assert os.path.exists(input_dir / 'grompp.mdp')

In [ ]:
# Figure 1b code.
grompp_input_files = {'-f': os.path.join(cmd_dir, 'grompp.mdp'),
                      '-c': make_top.output.file['-o'],
                      '-p': make_top.output.file['-p']}

# make array of inputs
N = 50
grompp = gmx.commandline_operation(
    'gmx', ['grompp'],
    input_files = [grompp_input_files] * N,
    output_files = {'-o': 'run.tpr'})
tpr_input = grompp.output.file['-o'].result()

input_list = gmx.read_tpr(tpr_input)

md = gmx.mdrun(input_list)
md.run()

print(md.output.trajectory.result())

## Figure 1c: looping and custom operations

In [ ]:
reference_struct = input_dir / 'ref.pdb'
assert reference_struct.exists()

In [ ]:
subgraph = gmx.subgraph(variables = {'found_native': False, 'checkpoint': '', 'min_rms': 1e6})
with subgraph:
    md = gmx.mdrun(input_list, runtime_args={'-cpi': subgraph.checkpoint, '-maxh': 2})
    subgraph.checkpoint = md.output.checkpoint
    rmsd = gmx.commandline_operation(
        'gmx', ['rms'],
        input_files={
            '-s': reference_struct,
            '-f': md.output.trajectory},
        output_files={'-o': 'rmsd.xvg'})
    subgraph.min_rms = numpy_min(xvg_parse(rmsd.output.file['-o'], [], keycol=0).output.data).output.value
    subgraph.found_native = less_than(lhs=subgraph.min_rms, rhs=0.3).output.data

folding_loop = gmx.while_loop(operation=subgraph, condition=subgraph.found_native)()
folding_loop.run()
